In [1]:
import os
os.chdir('../data')

os.getcwd()

'/Users/sagar/ms-iiit/is-music/data'

In [2]:
import glob
import json

from pydub import AudioSegment

In [3]:
DOWN_DATA_DIR = './yt8m-av-down-data'
CLIPS_DATA_DIR = './yt8m-audio-clips'
split = 'test'

In [4]:
threshold = 8000
clip_off_last = 3000
format = 'mp3'

In [5]:
input_split_dir = os.path.join(DOWN_DATA_DIR, split)
output_split_dir = os.path.join(CLIPS_DATA_DIR, split)
info_file_name = f'audio-clips-info-{split}.jsonl'

os.makedirs(output_split_dir, exist_ok=True)
os.system(f"touch ./{CLIPS_DATA_DIR}/{info_file_name}")

0

In [6]:
for vid in os.listdir(input_split_dir):
    if len(os.listdir(os.path.join(input_split_dir, vid))) != 2:
        continue
    audio_file = glob.glob(pathname=f'{input_split_dir}/{vid}/{vid}-audio.*')[0]
    audio_segment = AudioSegment.from_file(audio_file)
    dur_ms = len(audio_segment)
    dur_sec = len(audio_segment)/1000
    start = 0
    end = 0
    c = 0
    
    os.makedirs(os.path.join(output_split_dir, vid), exist_ok=True)
    while start < (dur_ms - clip_off_last):
        end += threshold
        clip = audio_segment[start:end]
        c += 1
        clip.export(os.path.join(output_split_dir, vid, f'{vid}-audio-{c}.{format}'), format=format)
        start += threshold
    info = {
        'vid': vid,
        'duration_sec': dur_sec,
        'n_clips': c,
        'last_clip_dur': len(clip)/1000.0
    }
    with open(os.path.join(CLIPS_DATA_DIR, info_file_name), 'a') as f:
        f.write(json.dumps(info) + '\n')